Since I will be working in an IDE and importing my functions and classes in this notebook, I can use the **autoreload** magic method that automatically reloads my code when I modify it, so I do not have to restart the notebook everytime.

**Notebook contains only the code to test implementation and the implementation itself is in the file.**

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload

In [50]:
import pandas as pd
from model import load_dataset, build_model
from transformers import CategoriesExtractor, GoalAdjustor, TimeTransformer, \
    CountryTransformer
pd.set_option('display.max_columns', 100)

In [8]:
X_train, y_train = load_dataset("X_train.zip", "y_train.zip")
X_train.head()

,photo,name,blurb,goal,slug,disable_communication,country,currency,currency_symbol,currency_trailing_code,deadline,created_at,launched_at,static_usd_rate,creator,location,category,profile,urls,source_url,friends,is_starred,is_backing,permissions
id,,,,,,,,,,,,,,,,,,,,,,,,
810643898,"{""small"":""https://ksr-ugc.imgix.net/assets/011...",Reverence: A Documentary Short on Branded Yarm...,A documentary exploring the phenomenon of cust...,6000.0,reverence-a-documentary-short-on-custom-yarmulkes,False,US,USD,$,True,1383676790,1378142506,1380217190,1.0,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""country"":""US"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...","{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN
407153952,"{""small"":""https://ksr-ugc.imgix.net/assets/011...",YOU Release the NEW Goddamn Electric Bill VINY...,Guarantee yourself one of 250 limited vinyl re...,1500.0,you-release-the-new-goddamn-electric-bill-viny...,False,US,USD,$,True,1284757200,1282000087,1282175023,1.0,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""country"":""US"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...","{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN
531190382,"{""small"":""https://ksr-ugc.imgix.net/assets/011...",Lonely Estates Album,Lonely Estates are almost done with their debu...,1000.0,lonely-estates-album,False,US,USD,$,True,1319133283,1315424966,1317318883,1.0,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""country"":""US"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...","{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN
1253528325,"{""small"":""https://ksr-ugc.imgix.net/assets/012...",Crit Hit! 2016 - a tabletop roleplaying gather...,A tabletop RPG focused event where you can pla...,3000.0,crit-hit-2016-a-tabletop-roleplaying-gathering-in,False,US,USD,$,True,1460442791,1451709060,1456990391,1.0,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""country"":""US"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":1,""should_show_fea...","{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN
379783411,"{""small"":""https://ksr-ugc.imgix.net/assets/011...",Hillsboro Arts Preschool Yearbook,"The Yearbook will be a collection of drawings,...",2500.0,hillsboro-arts-preschool-yearbook,False,US,USD,$,True,1356197572,1336233031,1353605572,1.0,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""country"":""US"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...","{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN,NaN,NaN,NaN


In [12]:
X_train.category[379783411]

'{"urls":{"web":{"discover":"http://www.kickstarter.com/discover/categories/publishing/art%20books"}},"color":14867664,"parent_id":18,"name":"Art Books","id":45,"position":3,"slug":"publishing/art books"}'

**Here we will create a new transformer that allows to extra categories from the `category` column that is stored as a json. We want this transformer to have a parameter `use_all` that allows us to choose between filtering to use only a subset of hardcoded categories we care about or get all categories found in the json.**

**The `transform` method should return two new columns, `gen_cat` the generic categorie, and `precise_cat` the precise category, those are extracted by assuming the json contains a string in the format `gen_cat/precise_cat`. We provide the method `_get_slug` that gets the two categories, and filter if necessary, so you only have to implement `fit` and `transform` to return the two new columns we want:**

In [37]:
ce = CategoriesExtractor(use_all=False)
ce.fit(X_train)
ce.transform(X_train).head()

,gen_cat,precise_cat
id,,
810643898,film & video,documentary
407153952,music,misc
531190382,music,rock
1253528325,games,misc
379783411,publishing,misc


In [18]:
X_train.columns

Index(['photo', 'name', 'blurb', 'goal', 'slug', 'disable_communication',
       'country', 'currency', 'currency_symbol', 'currency_trailing_code',
       'deadline', 'created_at', 'launched_at', 'static_usd_rate', 'creator',
       'location', 'category', 'profile', 'urls', 'source_url', 'friends',
       'is_starred', 'is_backing', 'permissions'],
      dtype='object')

**Simple transformer that returns column adjusted_goal which is the goal multiplied by static_usd_rate**

In [43]:
ga = GoalAdjustor()
ga.transform(X_train).head()

,adjusted_goal
id,
810643898,6000.0
407153952,1500.0
531190382,1000.0
1253528325,3000.0
379783411,2500.0


### Feature Engineering

**Computes number of days betweeen deadline and launched, launched and created.**

In [39]:
tt = TimeTransformer()
tt.transform(X_train).head()

,launched_to_deadline,created_to_launched
id,,
810643898,40,24
407153952,29,2
531190382,21,21
1253528325,39,61
379783411,30,201


### CountryTransformer
**Returns a larger area for the country feature, allowing to have less dummy features later. Provides a dictionary of countries and their corresponding groups.**

In [29]:
ct = CountryTransformer()
ct.transform(X_train).sample(10)

,country
id,
1017456318,US
529322416,US
184095173,US
87488242,US
1217484817,US
833025938,US
267225132,Oceania
1748542962,US
1997775757,Europe


## ColumnTransformer & Pipeline

**Here we will implement the `build_model` function that is defined in `model.py`. This function should return a new Pipeline object that has two stages:**
- `preprocessor`: A ColumnTransformer object that has all your preprocessing steps
- `model`: A predictive model, here we will use the `DecisionTreeClassifier`

**We are providing code to build two simple intermediary Pipeline objects: `cat_processor` and `country_processor`. Those are just combining our `CategoriesExtractor` and `CountryTransformer` with a `OneHotEncoder` stage so the output is an array of 1 and 0 for all.**

- the main ColumnTransformer that puts all the transformers together and applies them on the right columns
- the final Pipeline object that puts together the preprocessor and model.

The code below will get a new model using this function, train it on the data and generate predictions

In [52]:
model = build_model()
model.fit(X_train, y_train)
model.predict(X_train)

array([1, 1, 1, ..., 0, 1, 1])